In [1]:
import pandas as pd
import numpy as np

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
pd.set_option("display.max_columns", 999)

# Import Analysis Data

In [2]:
%%time
df = pd.read_csv("./input/dataReady.csv", index_col=0)
LabelMap = np.load('./input/LabelMapping.npy').item()

/usr/local/lib/python3.5/dist-packages/numpy/lib/arraysetops.py:466: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


CPU times: user 15.3 s, sys: 1.28 s, total: 16.6 s
Wall time: 16.8 s


In [3]:
%time df.replace(LabelMap, inplace=True)

CPU times: user 1min 14s, sys: 303 ms, total: 1min 14s
Wall time: 1min 14s


In [4]:
categorical_colName = ['from_station_name', 'usertype', 'gender', 'start_year', 'daily_icon', 'daily_precipType', 'daily_summary', 'icon', 'precipType', 'summary']

## Train Validation Test Split

I use 75%/25% as the training/testing set split.

In [88]:
from sklearn.model_selection import train_test_split

In [89]:
%%time
train, test = train_test_split(df, test_size=.25, train_size=.75, stratify=df['start_month'].tolist(), random_state=0)

CPU times: user 3.54 s, sys: 800 ms, total: 4.34 s
Wall time: 5.28 s


In [90]:
print(train.shape, test.shape)

(2757816, 61) (919272, 61)


## LightGBM Model Parameter Tuning

I use [LightGBM](https://github.com/Microsoft/LightGBM) as the first model. LightGBM is a fast, distributed, high performance gradient boosting framework based on decision tree algorithms, used for ranking, classification and many other machine learning tasks. I use 5-fold cross validation to tune the parameters at first.

In [91]:
from sklearn.model_selection import GridSearchCV

In [92]:
# create input data
def generate_model_input(df):
    '''return format: X_train, y_train'''
    keep_col = df.columns.tolist()
    keep_col.remove('distance')
    
    return df[keep_col], (df['distance'] < 2.0).tolist()

X_train, y_train = generate_model_input(train)
X_test, y_test = generate_model_input(test)

# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

In [61]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'binary_logloss', 'auc'},
    'num_boost_round': 500,
    'learning_rate': .1,
    'num_leaves': 31,
    'max_bin': 255,
    'lambda_l2': .05,
    'num_threads':8
}

param_grid = {
    'boosting_type': ['gbdt'],
    'objective': ['binary'],
    'metric': ['binary_logloss'],
    'num_boost_round': [500],
    'learning_rate': [.1],
    'num_leaves': [20,30,50,70,90],
    'max_bin': [255,383,511],
    'lambda_l2': [0, .05, .1],
    'num_threads': [8]
}

In [62]:
mod = lgb.LGBMRegressor(boosting_type=params['boosting_type'],
                        objective=params['objective'],
                        metric=params['metric'],
                        num_boost_round=params['num_boost_round'],
                        learning_rate=params['learning_rate'],
                        num_leaves=params['num_leaves'],
                        max_bin=params['max_bin'],
                        lambda_l2=params['lambda_l2'],
                        num_threads=params['num_threads']
                       )

In [63]:
%%time
print('CV training start...')
grid = GridSearchCV(mod, param_grid, cv=5)
grid.fit(X_train, y_train)

CV training start...


/usr/local/lib/python3.5/dist-packages/lightgbm/engine.py:99: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.5/dist-packages/lightgbm/engine.py:99: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.5/dist-packages/lightgbm/engine.py:99: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.5/dist-packages/lightgbm/engine.py:99: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.5/dist-packages/lightgbm/engine.py:99: UserWarning: Found `nu

/usr/local/lib/python3.5/dist-packages/lightgbm/engine.py:99: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.5/dist-packages/lightgbm/engine.py:99: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.5/dist-packages/lightgbm/engine.py:99: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.5/dist-packages/lightgbm/engine.py:99: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.5/dist-packages/lightgbm/engine.py:99: UserWarning: Found `nu

/usr/local/lib/python3.5/dist-packages/lightgbm/engine.py:99: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.5/dist-packages/lightgbm/engine.py:99: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.5/dist-packages/lightgbm/engine.py:99: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.5/dist-packages/lightgbm/engine.py:99: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.5/dist-packages/lightgbm/engine.py:99: UserWarning: Found `nu

/usr/local/lib/python3.5/dist-packages/lightgbm/engine.py:99: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.5/dist-packages/lightgbm/engine.py:99: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.5/dist-packages/lightgbm/engine.py:99: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.5/dist-packages/lightgbm/engine.py:99: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.5/dist-packages/lightgbm/engine.py:99: UserWarning: Found `nu

/usr/local/lib/python3.5/dist-packages/lightgbm/engine.py:99: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.5/dist-packages/lightgbm/engine.py:99: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.5/dist-packages/lightgbm/engine.py:99: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.5/dist-packages/lightgbm/engine.py:99: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.5/dist-packages/lightgbm/engine.py:99: UserWarning: Found `nu

/usr/local/lib/python3.5/dist-packages/lightgbm/engine.py:99: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.5/dist-packages/lightgbm/engine.py:99: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.5/dist-packages/lightgbm/engine.py:99: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.5/dist-packages/lightgbm/engine.py:99: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.5/dist-packages/lightgbm/engine.py:99: UserWarning: Found `nu

/usr/local/lib/python3.5/dist-packages/lightgbm/engine.py:99: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.5/dist-packages/lightgbm/engine.py:99: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.5/dist-packages/lightgbm/engine.py:99: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.5/dist-packages/lightgbm/engine.py:99: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.5/dist-packages/lightgbm/engine.py:99: UserWarning: Found `nu

CPU times: user 1d 8h 26min 2s, sys: 12min 37s, total: 1d 8h 38min 39s
Wall time: 4h 38min 56s


In [64]:
grid.best_params_

{'boosting_type': 'gbdt',
 'lambda_l2': 0,
 'learning_rate': 0.1,
 'max_bin': 383,
 'metric': 'binary_logloss',
 'num_boost_round': 500,
 'num_leaves': 90,
 'num_threads': 8,
 'objective': 'binary'}

As observed above, the best model from the 5-fold cross validation has the above parameter settings.

## Microsoft LightGBM Model

Using the parameter tunned by 5-fold cross validation, I fit LightGBM model with learning rate = .05 for 10,000 rounds.

In [93]:
import lightgbm as lgb

In [94]:
categorical_colName

['from_station_name',
 'usertype',
 'gender',
 'start_year',
 'daily_icon',
 'daily_precipType',
 'daily_summary',
 'icon',
 'precipType',
 'summary']

In [95]:
%%time
# specify your configurations as a dict
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'binary_logloss', 'auc'},
    'num_leaves': 90,
    'max_bin': 383,
    'learning_rate': 0.05,
    'verbose': 0,
    'num_threads':8
}

print('Start training...')
# train
gbm = lgb.train(params,
                lgb_train,
                categorical_feature = categorical_colName,
                num_boost_round=10000,
                valid_sets=lgb_eval)

print('Save model...')
# save model to file
gbm.save_model('lgb_model_1.txt')

Start training...


/usr/local/lib/python3.5/dist-packages/lightgbm/basic.py:1029: UserWarning: categorical_feature in Dataset is overrided. New categorical_feature is ['daily_icon', 'daily_precipType', 'daily_summary', 'from_station_name', 'gender', 'icon', 'precipType', 'start_year', 'summary', 'usertype']
  warnings.warn('categorical_feature in Dataset is overrided. New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/usr/local/lib/python3.5/dist-packages/lightgbm/basic.py:668: UserWarning: categorical_feature in param dict is overrided.
  warnings.warn('categorical_feature in param dict is overrided.')


[1]	valid_0's binary_logloss: 0.684583	valid_0's auc: 0.699021
[2]	valid_0's binary_logloss: 0.676821	valid_0's auc: 0.700697
[3]	valid_0's binary_logloss: 0.669777	valid_0's auc: 0.701333
[4]	valid_0's binary_logloss: 0.663394	valid_0's auc: 0.70195
[5]	valid_0's binary_logloss: 0.657633	valid_0's auc: 0.702945
[6]	valid_0's binary_logloss: 0.652376	valid_0's auc: 0.703847
[7]	valid_0's binary_logloss: 0.647647	valid_0's auc: 0.704063
[8]	valid_0's binary_logloss: 0.64317	valid_0's auc: 0.704678
[9]	valid_0's binary_logloss: 0.639037	valid_0's auc: 0.705237
[10]	valid_0's binary_logloss: 0.635371	valid_0's auc: 0.705398
[11]	valid_0's binary_logloss: 0.631856	valid_0's auc: 0.706109
[12]	valid_0's binary_logloss: 0.628703	valid_0's auc: 0.706381
[13]	valid_0's binary_logloss: 0.625811	valid_0's auc: 0.706688
[14]	valid_0's binary_logloss: 0.623129	valid_0's auc: 0.706857
[15]	valid_0's binary_logloss: 0.620656	valid_0's auc: 0.707233
[16]	valid_0's binary_logloss: 0.618405	valid_0's a

[130]	valid_0's binary_logloss: 0.572747	valid_0's auc: 0.736743
[131]	valid_0's binary_logloss: 0.572627	valid_0's auc: 0.736901
[132]	valid_0's binary_logloss: 0.572601	valid_0's auc: 0.736931
[133]	valid_0's binary_logloss: 0.572477	valid_0's auc: 0.737082
[134]	valid_0's binary_logloss: 0.572439	valid_0's auc: 0.737131
[135]	valid_0's binary_logloss: 0.57233	valid_0's auc: 0.737279
[136]	valid_0's binary_logloss: 0.572209	valid_0's auc: 0.737434
[137]	valid_0's binary_logloss: 0.572092	valid_0's auc: 0.737583
[138]	valid_0's binary_logloss: 0.572047	valid_0's auc: 0.737648
[139]	valid_0's binary_logloss: 0.571984	valid_0's auc: 0.737732
[140]	valid_0's binary_logloss: 0.571876	valid_0's auc: 0.737871
[141]	valid_0's binary_logloss: 0.571854	valid_0's auc: 0.737895
[142]	valid_0's binary_logloss: 0.571779	valid_0's auc: 0.737986
[143]	valid_0's binary_logloss: 0.571635	valid_0's auc: 0.73818
[144]	valid_0's binary_logloss: 0.571544	valid_0's auc: 0.738301
[145]	valid_0's binary_logl

[258]	valid_0's binary_logloss: 0.565209	valid_0's auc: 0.746242
[259]	valid_0's binary_logloss: 0.565192	valid_0's auc: 0.74626
[260]	valid_0's binary_logloss: 0.565185	valid_0's auc: 0.746268
[261]	valid_0's binary_logloss: 0.565175	valid_0's auc: 0.746281
[262]	valid_0's binary_logloss: 0.565168	valid_0's auc: 0.746289
[263]	valid_0's binary_logloss: 0.565145	valid_0's auc: 0.746317
[264]	valid_0's binary_logloss: 0.565133	valid_0's auc: 0.746331
[265]	valid_0's binary_logloss: 0.565118	valid_0's auc: 0.746351
[266]	valid_0's binary_logloss: 0.565082	valid_0's auc: 0.7464
[267]	valid_0's binary_logloss: 0.565062	valid_0's auc: 0.746429
[268]	valid_0's binary_logloss: 0.565051	valid_0's auc: 0.74644
[269]	valid_0's binary_logloss: 0.565038	valid_0's auc: 0.746455
[270]	valid_0's binary_logloss: 0.56502	valid_0's auc: 0.746476
[271]	valid_0's binary_logloss: 0.565014	valid_0's auc: 0.746485
[272]	valid_0's binary_logloss: 0.565012	valid_0's auc: 0.746486
[273]	valid_0's binary_logloss

...

[9425]	valid_0's binary_logloss: 0.5317	valid_0's auc: 0.782801
[9426]	valid_0's binary_logloss: 0.531696	valid_0's auc: 0.782804
[9427]	valid_0's binary_logloss: 0.531697	valid_0's auc: 0.782804
[9428]	valid_0's binary_logloss: 0.531691	valid_0's auc: 0.782809
[9429]	valid_0's binary_logloss: 0.531691	valid_0's auc: 0.782809
[9430]	valid_0's binary_logloss: 0.531689	valid_0's auc: 0.782811
[9431]	valid_0's binary_logloss: 0.53169	valid_0's auc: 0.78281
[9432]	valid_0's binary_logloss: 0.53169	valid_0's auc: 0.782811
[9433]	valid_0's binary_logloss: 0.531689	valid_0's auc: 0.782811
[9434]	valid_0's binary_logloss: 0.531676	valid_0's auc: 0.782822
[9435]	valid_0's binary_logloss: 0.531671	valid_0's auc: 0.782828
[9436]	valid_0's binary_logloss: 0.531671	valid_0's auc: 0.782827
[9437]	valid_0's binary_logloss: 0.53167	valid_0's auc: 0.782828
[9438]	valid_0's binary_logloss: 0.53167	valid_0's auc: 0.782828
[9439]	valid_0's binary_logloss: 0.531665	valid_0's auc: 0.782833
[9440]	valid_0's 

[9550]	valid_0's binary_logloss: 0.531564	valid_0's auc: 0.782944
[9551]	valid_0's binary_logloss: 0.531563	valid_0's auc: 0.782945
[9552]	valid_0's binary_logloss: 0.531564	valid_0's auc: 0.782945
[9553]	valid_0's binary_logloss: 0.531565	valid_0's auc: 0.782945
[9554]	valid_0's binary_logloss: 0.531562	valid_0's auc: 0.782948
[9555]	valid_0's binary_logloss: 0.531562	valid_0's auc: 0.782948
[9556]	valid_0's binary_logloss: 0.531561	valid_0's auc: 0.782949
[9557]	valid_0's binary_logloss: 0.531562	valid_0's auc: 0.782948
[9558]	valid_0's binary_logloss: 0.531562	valid_0's auc: 0.782948
[9559]	valid_0's binary_logloss: 0.531565	valid_0's auc: 0.782945
[9560]	valid_0's binary_logloss: 0.531565	valid_0's auc: 0.782946
[9561]	valid_0's binary_logloss: 0.531565	valid_0's auc: 0.782945
[9562]	valid_0's binary_logloss: 0.531565	valid_0's auc: 0.782946
[9563]	valid_0's binary_logloss: 0.531563	valid_0's auc: 0.782948
[9564]	valid_0's binary_logloss: 0.531565	valid_0's auc: 0.782946
[9565]	val

[9675]	valid_0's binary_logloss: 0.531232	valid_0's auc: 0.783263
[9676]	valid_0's binary_logloss: 0.531234	valid_0's auc: 0.783262
[9677]	valid_0's binary_logloss: 0.531235	valid_0's auc: 0.78326
[9678]	valid_0's binary_logloss: 0.531232	valid_0's auc: 0.783263
[9679]	valid_0's binary_logloss: 0.531233	valid_0's auc: 0.783263
[9680]	valid_0's binary_logloss: 0.531231	valid_0's auc: 0.783265
[9681]	valid_0's binary_logloss: 0.531232	valid_0's auc: 0.783264
[9682]	valid_0's binary_logloss: 0.531232	valid_0's auc: 0.783264
[9683]	valid_0's binary_logloss: 0.531231	valid_0's auc: 0.783266
[9684]	valid_0's binary_logloss: 0.531221	valid_0's auc: 0.783276
[9685]	valid_0's binary_logloss: 0.531222	valid_0's auc: 0.783274
[9686]	valid_0's binary_logloss: 0.531221	valid_0's auc: 0.783276
[9687]	valid_0's binary_logloss: 0.531222	valid_0's auc: 0.783275
[9688]	valid_0's binary_logloss: 0.531223	valid_0's auc: 0.783274
[9689]	valid_0's binary_logloss: 0.531217	valid_0's auc: 0.783281
[9690]	vali

[9800]	valid_0's binary_logloss: 0.531181	valid_0's auc: 0.783328
[9801]	valid_0's binary_logloss: 0.531181	valid_0's auc: 0.783328
[9802]	valid_0's binary_logloss: 0.531179	valid_0's auc: 0.783331
[9803]	valid_0's binary_logloss: 0.531178	valid_0's auc: 0.783332
[9804]	valid_0's binary_logloss: 0.531177	valid_0's auc: 0.783333
[9805]	valid_0's binary_logloss: 0.531178	valid_0's auc: 0.783332
[9806]	valid_0's binary_logloss: 0.531178	valid_0's auc: 0.783332
[9807]	valid_0's binary_logloss: 0.531179	valid_0's auc: 0.78333
[9808]	valid_0's binary_logloss: 0.531179	valid_0's auc: 0.783331
[9809]	valid_0's binary_logloss: 0.531179	valid_0's auc: 0.783331
[9810]	valid_0's binary_logloss: 0.531179	valid_0's auc: 0.783331
[9811]	valid_0's binary_logloss: 0.531178	valid_0's auc: 0.783332
[9812]	valid_0's binary_logloss: 0.53118	valid_0's auc: 0.783331
[9813]	valid_0's binary_logloss: 0.53118	valid_0's auc: 0.78333
[9814]	valid_0's binary_logloss: 0.531179	valid_0's auc: 0.78333
[9815]	valid_0'

[9926]	valid_0's binary_logloss: 0.531069	valid_0's auc: 0.783456
[9927]	valid_0's binary_logloss: 0.531062	valid_0's auc: 0.783464
[9928]	valid_0's binary_logloss: 0.531061	valid_0's auc: 0.783465
[9929]	valid_0's binary_logloss: 0.531059	valid_0's auc: 0.783467
[9930]	valid_0's binary_logloss: 0.531059	valid_0's auc: 0.783467
[9931]	valid_0's binary_logloss: 0.531058	valid_0's auc: 0.783468
[9932]	valid_0's binary_logloss: 0.531059	valid_0's auc: 0.783467
[9933]	valid_0's binary_logloss: 0.531058	valid_0's auc: 0.783469
[9934]	valid_0's binary_logloss: 0.531058	valid_0's auc: 0.783469
[9935]	valid_0's binary_logloss: 0.531059	valid_0's auc: 0.78347
[9936]	valid_0's binary_logloss: 0.531058	valid_0's auc: 0.783471
[9937]	valid_0's binary_logloss: 0.531059	valid_0's auc: 0.78347
[9938]	valid_0's binary_logloss: 0.531059	valid_0's auc: 0.78347
[9939]	valid_0's binary_logloss: 0.531058	valid_0's auc: 0.783472
[9940]	valid_0's binary_logloss: 0.531056	valid_0's auc: 0.783473
[9941]	valid_

In [78]:
# predict
y_pred_gbm = gbm.predict(X_test, num_iteration=gbm.best_iteration)

Start predicting...
Test accuracy is: 0.7212714401877571


# Xgboost Model

I also fit a [Xgboost](https://github.com/dmlc/xgboost) model. I will fit ensemble model later using both models.

In [13]:
import xgboost as xgb

In [14]:
%%time
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)
watchlist = [(dtrain,'train'), (dvalid,'eval')]

CPU times: user 1.52 s, sys: 1.49 s, total: 3.01 s
Wall time: 3.01 s


In [17]:
xparams = {'objective':'binary:logistic',
            'eval_metric': 'logloss',
            'eta': .05,
            'max_depth':7,
            'silent':0,
            'predictor': 'cpu_predictor'}

%time clf = xgb.train(xparams, dtrain, 4000, watchlist, verbose_eval=1)

[0]	train-logloss:0.68557	eval-logloss:0.68558
[1]	train-logloss:0.678732	eval-logloss:0.678758
[2]	train-logloss:0.672472	eval-logloss:0.672509
[3]	train-logloss:0.666803	eval-logloss:0.666858
[4]	train-logloss:0.661559	eval-logloss:0.66163
[5]	train-logloss:0.656706	eval-logloss:0.656788
[6]	train-logloss:0.652309	eval-logloss:0.652398
[7]	train-logloss:0.648384	eval-logloss:0.648485
[8]	train-logloss:0.644748	eval-logloss:0.644849
[9]	train-logloss:0.64131	eval-logloss:0.641413
[10]	train-logloss:0.638226	eval-logloss:0.63834
[11]	train-logloss:0.635361	eval-logloss:0.635486
[12]	train-logloss:0.632791	eval-logloss:0.632926
[13]	train-logloss:0.630483	eval-logloss:0.63063
[14]	train-logloss:0.627948	eval-logloss:0.628098
[15]	train-logloss:0.625632	eval-logloss:0.625788
[16]	train-logloss:0.623489	eval-logloss:0.623656
[17]	train-logloss:0.621535	eval-logloss:0.62171
[18]	train-logloss:0.619736	eval-logloss:0.619923
[19]	train-logloss:0.618193	eval-logloss:0.618391
[20]	train-loglos

[164]	train-logloss:0.585223	eval-logloss:0.587037
[165]	train-logloss:0.585159	eval-logloss:0.586992
[166]	train-logloss:0.585098	eval-logloss:0.586939
[167]	train-logloss:0.585075	eval-logloss:0.586926
[168]	train-logloss:0.585016	eval-logloss:0.586871
[169]	train-logloss:0.584977	eval-logloss:0.586836
[170]	train-logloss:0.584939	eval-logloss:0.586815
[171]	train-logloss:0.584902	eval-logloss:0.586781
[172]	train-logloss:0.584853	eval-logloss:0.586737
[173]	train-logloss:0.584821	eval-logloss:0.586708
[174]	train-logloss:0.584715	eval-logloss:0.586617
[175]	train-logloss:0.584672	eval-logloss:0.586588
[176]	train-logloss:0.584634	eval-logloss:0.58656
[177]	train-logloss:0.58458	eval-logloss:0.586508
[178]	train-logloss:0.584514	eval-logloss:0.58645
[179]	train-logloss:0.584481	eval-logloss:0.586419
[180]	train-logloss:0.584369	eval-logloss:0.586318
[181]	train-logloss:0.584302	eval-logloss:0.58627
[182]	train-logloss:0.58422	eval-logloss:0.586201
[183]	train-logloss:0.584106	eval-lo

[326]	train-logloss:0.576288	eval-logloss:0.579681
[327]	train-logloss:0.576258	eval-logloss:0.579659
[328]	train-logloss:0.57624	eval-logloss:0.579651
[329]	train-logloss:0.576123	eval-logloss:0.579542
[330]	train-logloss:0.575957	eval-logloss:0.579396
[331]	train-logloss:0.575899	eval-logloss:0.579345
[332]	train-logloss:0.575865	eval-logloss:0.579325
[333]	train-logloss:0.575847	eval-logloss:0.579315
[334]	train-logloss:0.575805	eval-logloss:0.579286
[335]	train-logloss:0.57568	eval-logloss:0.579168
[336]	train-logloss:0.575616	eval-logloss:0.579112
[337]	train-logloss:0.575591	eval-logloss:0.579094
[338]	train-logloss:0.575534	eval-logloss:0.579044
[339]	train-logloss:0.575518	eval-logloss:0.579039
[340]	train-logloss:0.575497	eval-logloss:0.579024
[341]	train-logloss:0.575433	eval-logloss:0.578966
[342]	train-logloss:0.575376	eval-logloss:0.578915
[343]	train-logloss:0.575327	eval-logloss:0.578878
[344]	train-logloss:0.575188	eval-logloss:0.578746
[345]	train-logloss:0.575127	eval

[488]	train-logloss:0.570102	eval-logloss:0.575053
[489]	train-logloss:0.570025	eval-logloss:0.574982
[490]	train-logloss:0.569987	eval-logloss:0.574957
[491]	train-logloss:0.56995	eval-logloss:0.574928
[492]	train-logloss:0.569946	eval-logloss:0.574928
[493]	train-logloss:0.569924	eval-logloss:0.574921
[494]	train-logloss:0.569917	eval-logloss:0.574916
[495]	train-logloss:0.569906	eval-logloss:0.574908
[496]	train-logloss:0.569896	eval-logloss:0.574907
[497]	train-logloss:0.569874	eval-logloss:0.574887
[498]	train-logloss:0.569861	eval-logloss:0.574881
[499]	train-logloss:0.569817	eval-logloss:0.574849
[500]	train-logloss:0.569717	eval-logloss:0.57476
[501]	train-logloss:0.56968	eval-logloss:0.574731
[502]	train-logloss:0.569613	eval-logloss:0.574671
[503]	train-logloss:0.56958	eval-logloss:0.574646
[504]	train-logloss:0.569477	eval-logloss:0.574554
[505]	train-logloss:0.569459	eval-logloss:0.574541
[506]	train-logloss:0.569426	eval-logloss:0.574517
[507]	train-logloss:0.569407	eval-l

[650]	train-logloss:0.564954	eval-logloss:0.571375
[651]	train-logloss:0.564937	eval-logloss:0.571367
[652]	train-logloss:0.564932	eval-logloss:0.571366
[653]	train-logloss:0.564908	eval-logloss:0.571355
[654]	train-logloss:0.564903	eval-logloss:0.571354
[655]	train-logloss:0.564883	eval-logloss:0.571343
[656]	train-logloss:0.564843	eval-logloss:0.571317
[657]	train-logloss:0.56483	eval-logloss:0.571314
[658]	train-logloss:0.564814	eval-logloss:0.571311
[659]	train-logloss:0.56481	eval-logloss:0.571311
[660]	train-logloss:0.564785	eval-logloss:0.571297
[661]	train-logloss:0.564763	eval-logloss:0.571288
[662]	train-logloss:0.564726	eval-logloss:0.571267
[663]	train-logloss:0.564714	eval-logloss:0.571263
[664]	train-logloss:0.564703	eval-logloss:0.571258
[665]	train-logloss:0.564687	eval-logloss:0.571256
[666]	train-logloss:0.564664	eval-logloss:0.571246
[667]	train-logloss:0.564653	eval-logloss:0.571243
[668]	train-logloss:0.564647	eval-logloss:0.57124
[669]	train-logloss:0.564644	eval-

...

[3358]	train-logloss:0.524705	eval-logloss:0.55359
[3359]	train-logloss:0.524697	eval-logloss:0.55359
[3360]	train-logloss:0.524682	eval-logloss:0.553586
[3361]	train-logloss:0.524663	eval-logloss:0.553576
[3362]	train-logloss:0.524651	eval-logloss:0.553572
[3363]	train-logloss:0.524642	eval-logloss:0.553573
[3364]	train-logloss:0.524642	eval-logloss:0.553573
[3365]	train-logloss:0.524641	eval-logloss:0.553573
[3366]	train-logloss:0.524623	eval-logloss:0.553564
[3367]	train-logloss:0.524618	eval-logloss:0.553563
[3368]	train-logloss:0.524613	eval-logloss:0.553562
[3369]	train-logloss:0.524613	eval-logloss:0.553562
[3370]	train-logloss:0.524612	eval-logloss:0.553562
[3371]	train-logloss:0.524611	eval-logloss:0.553562
[3372]	train-logloss:0.524606	eval-logloss:0.553562
[3373]	train-logloss:0.5246	eval-logloss:0.553562
[3374]	train-logloss:0.524599	eval-logloss:0.553562
[3375]	train-logloss:0.524595	eval-logloss:0.553562
[3376]	train-logloss:0.524586	eval-logloss:0.553562
[3377]	train-log

[3517]	train-logloss:0.523404	eval-logloss:0.553414
[3518]	train-logloss:0.523378	eval-logloss:0.5534
[3519]	train-logloss:0.523362	eval-logloss:0.553397
[3520]	train-logloss:0.523346	eval-logloss:0.553393
[3521]	train-logloss:0.523337	eval-logloss:0.553389
[3522]	train-logloss:0.52333	eval-logloss:0.55339
[3523]	train-logloss:0.523324	eval-logloss:0.55339
[3524]	train-logloss:0.523317	eval-logloss:0.553391
[3525]	train-logloss:0.523303	eval-logloss:0.553389
[3526]	train-logloss:0.523295	eval-logloss:0.553388
[3527]	train-logloss:0.523262	eval-logloss:0.553366
[3528]	train-logloss:0.523254	eval-logloss:0.553364
[3529]	train-logloss:0.523245	eval-logloss:0.553364
[3530]	train-logloss:0.523238	eval-logloss:0.553364
[3531]	train-logloss:0.523223	eval-logloss:0.553361
[3532]	train-logloss:0.523215	eval-logloss:0.55336
[3533]	train-logloss:0.523206	eval-logloss:0.55336
[3534]	train-logloss:0.5232	eval-logloss:0.553358
[3535]	train-logloss:0.523195	eval-logloss:0.553357
[3536]	train-logloss:

[3676]	train-logloss:0.521796	eval-logloss:0.553078
[3677]	train-logloss:0.521781	eval-logloss:0.553073
[3678]	train-logloss:0.521777	eval-logloss:0.553073
[3679]	train-logloss:0.521775	eval-logloss:0.553073
[3680]	train-logloss:0.521763	eval-logloss:0.553072
[3681]	train-logloss:0.521741	eval-logloss:0.553059
[3682]	train-logloss:0.521719	eval-logloss:0.553049
[3683]	train-logloss:0.521695	eval-logloss:0.553032
[3684]	train-logloss:0.521666	eval-logloss:0.553011
[3685]	train-logloss:0.521647	eval-logloss:0.553003
[3686]	train-logloss:0.521612	eval-logloss:0.552977
[3687]	train-logloss:0.521602	eval-logloss:0.552976
[3688]	train-logloss:0.521593	eval-logloss:0.552976
[3689]	train-logloss:0.521583	eval-logloss:0.552974
[3690]	train-logloss:0.521575	eval-logloss:0.552972
[3691]	train-logloss:0.521559	eval-logloss:0.552964
[3692]	train-logloss:0.521551	eval-logloss:0.552964
[3693]	train-logloss:0.521537	eval-logloss:0.552961
[3694]	train-logloss:0.521529	eval-logloss:0.55296
[3695]	train-

[3835]	train-logloss:0.520096	eval-logloss:0.552572
[3836]	train-logloss:0.520084	eval-logloss:0.552574
[3837]	train-logloss:0.520073	eval-logloss:0.55257
[3838]	train-logloss:0.520063	eval-logloss:0.55257
[3839]	train-logloss:0.52006	eval-logloss:0.55257
[3840]	train-logloss:0.520049	eval-logloss:0.55257
[3841]	train-logloss:0.520038	eval-logloss:0.552565
[3842]	train-logloss:0.520024	eval-logloss:0.552563
[3843]	train-logloss:0.520016	eval-logloss:0.552563
[3844]	train-logloss:0.52001	eval-logloss:0.552562
[3845]	train-logloss:0.519998	eval-logloss:0.552562
[3846]	train-logloss:0.51999	eval-logloss:0.552561
[3847]	train-logloss:0.519978	eval-logloss:0.552559
[3848]	train-logloss:0.51997	eval-logloss:0.552559
[3849]	train-logloss:0.519958	eval-logloss:0.552555
[3850]	train-logloss:0.51995	eval-logloss:0.552553
[3851]	train-logloss:0.519938	eval-logloss:0.552553
[3852]	train-logloss:0.519929	eval-logloss:0.552553
[3853]	train-logloss:0.519918	eval-logloss:0.552553
[3854]	train-logloss:

[3994]	train-logloss:0.518619	eval-logloss:0.552246
[3995]	train-logloss:0.518611	eval-logloss:0.552246
[3996]	train-logloss:0.518601	eval-logloss:0.552245
[3997]	train-logloss:0.518593	eval-logloss:0.552245
[3998]	train-logloss:0.518587	eval-logloss:0.552244
[3999]	train-logloss:0.51858	eval-logloss:0.552244
CPU times: user 2d 7h 43min 26s, sys: 14.7 s, total: 2d 7h 43min 41s
Wall time: 7h 12min 49s


In [18]:
clf.save_model('xgb_model')